# OP2 Demo

Most people are comfortable with the F06.  However, it's:
 - a lot harder to parse
 - much, much, much slower
 - much, much more memory inefficient

The pyNastran OP2 Reader is fast, highly validated, and it supports most result types.

## Validating an OP2

The ```test_op2``` script is created when you run ```python setup.py develop``` or ```python setup.py install``` on pyNastran.  Assuming it's on your path (it'll be in Python27\Scripts or something similar), you can run:

```
>>> test_op2 -f solid_bending.op2
```
The ```-f``` tells us to print out ```solid_bending.test_op2.f06```, which can be compared to your F06 for a small file to build confidence in the reader.  It's also useful when you want an F06 of your model without rerunning Nastran just to see what's in it.

If you have a large model, you can make ```test_op2``` run much, much faster.  The ```-c``` flag disables double-reading of the OP2.  By default, ```test_op2``` uses two different read methods (the old method and new method) to ensure that results are read in properly.  When running the code, this is turned off, but is turned on for ```test_op2```.
```
>>> test_op2 -fc solid_bending.op2
```

## Import the packages

In [1]:
import os
import copy
import numpy as np

import pyNastran
pkg_path = pyNastran.__path__[0]

from pyNastran.utils import print_bad_path
from pyNastran.op2.op2 import read_op2
from pyNastran.utils import object_methods, object_attributes

import pandas as pd

### Sets default precision of real numbers for pandas output


In [2]:
pd.set_option('precision', 3)
np.set_printoptions(precision=3, threshold=20)

As with the BDF, we can use the long form and the short form.  However, the long form for the ``OP2`` doesn't really add anything.  So, let's just use the short form.

Besides massive speed improvements in the OP2 relative to v0.7, this version adds ```pandas``` dataframe support.

In [3]:
#op2_filename = r'D:\work\pynastran_0.8.0\models\iSat\ISat_Launch_Sm_Rgd.op2'
#op2_filename = r'D:\work\pynastran_0.8.0\models\iSat\ISat_Launch_Sm_4pt.op2'
op2_filename = os.path.abspath(os.path.join(pkg_path, '..', 'models', 'iSat', 'ISat_Launch_Sm_4pt.op2'))
assert os.path.exists(op2_filename), print_bad_path(op2_filename)

# define the input file with a file path
op2 = read_op2(op2_filename, build_dataframe=True, debug=False)

INFO:      fname=op2_scalar.pyc            lineNo=1020   op2_filename = 'f:\\work\\pynastran\\pynastran\\master3\\models\\iSat\\ISat_Launch_Sm_4pt.op2'


## OP2 Introspection
The ```get_op2_stats()``` function lets you quickly understand what in an op2.

In [4]:
print(op2.get_op2_stats())

eigenvectors[1]
  isubcase = 1
  type=RealEigenvectorArray ntimes=167 nnodes=5379
  data: [t1, t2, t3, r1, r2, r3] shape=[167, 5379, 6] dtype=float32
  gridTypes
  sort1
  modes = [  1   2   3 ..., 165 166 167]
  eigrs = [  2.758e+03   3.568e+03   9.686e+03 ...,   6.163e+06   6.170e+06
   6.230e+06]
  mode_cycles = [1112674317 1114566525 1120195719 ..., 1159407589 1159413465 1159462558]

cbar_force[1]
  type=RealCBarForceArray ntimes=167 nelements=827
  data: [ntimes, nnodes, 8] where 8=[bending_moment_a1, bending_moment_a2, bending_moment_b1, bending_moment_b2, shear1, shear2, axial, torque]
  data.shape = (167, 827, 8)
  element name: CBAR-34
  sort1
  modes = [  1   2   3 ..., 165 166 167]
  eigrs = [  2.758e+03   3.568e+03   9.686e+03 ...,   6.163e+06   6.170e+06
   6.230e+06]

ctria3_stress[1]
  type=RealPlateStressArray ntimes=167 nelements=32 nnodes_per_element=1 nlayers=2 ntotal=64
  data: [ntimes, ntotal, 8] where 8=[fiber_distance, oxx, oyy, txy, angle, omax, omin, von_mises]

### If that's too long...

In [5]:
print(op2.get_op2_stats(short=True))

eigenvectors[1]
cbar_force[1]
ctria3_stress[1]
cquad4_stress[1]
eigenvalues[u'ISAT_SM_LAUNCH_4PT MODES TO 400 HZ']



## Acccessing the Eigenvectors object

Eigenvectors are the simplest object.  They use the same class as for displacements, velocity, acceleration, SPC Forces, MPC Forces, Applied Loads, etc.  These are all node-based tables with TX, TY, TZ, RX, RY, RZ.  Results are in the analysis coordinate frame (CD), which is defined by the GRID card.

### Numpy-based Approach
We'll first show off the standard ```numpy``` based results on a transient case.  Static results are the same, except that you'll always use the 0th index for the "time" index.

The tutorial is intetionally just accessing the objects in a very clear, though inefficient way.  The OP2 objects can take full advantage of the numpy operations.

In [6]:
# what modes did we analyze:  1 to 167
print("loadcases = %s" % op2.eigenvectors.keys())

# get subcase 1
eig1 = op2.eigenvectors[1]

modes = eig1.modes
times = eig1._times #  the generic version of modes
print("modes = %s\n" % modes)
print("times = %s\n" % times)

imode2 = 1 # corresponds to mode 2
mode2 = eig1.data[imode2, :, :]

print('first 10 nodes and grid types\nNid Gridtype\n%s' % eig1.node_gridtype[:10, :])
node_ids = eig1.node_gridtype[:, 0]

index_node10 = np.where(node_ids == 10)[0]  # we add the [0] because it's 1d
mode2_node10 = mode2[index_node10]
print("translation mode2_node10 = %s" % eig1.data[imode2, index_node10, :3].ravel())
print("rotations mode2_node10 = %s" % eig1.data[imode2, index_node10, 3:].ravel())

loadcases = [1]
modes = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167]

times = [   1.    2.    3. ...,  165.  166.  167.]

first 10 nodes and grid types
Nid Gridtype
[[ 1  1]
 [ 2  1]
 [ 3  1]
 [ 4  1]
 [ 5  1]
 [ 6  1]
 [ 7  1]
 [ 8  1]
 [ 9  1]
 [10  1]]
translation mode2_node10 = [  1.696e-05   7.937e-03   1.510e-

### Pandas-based Approach

If you like pandas, you can access all the OP2 objects, which is very useful within the Jupyter Notebook.  Different objects will look differently, but you can change the layout.

If you're trying to learn pandas, there are many tutorials online, such as:
http://pandas.pydata.org/pandas-docs/stable/10min.html

or a very long, but good video:

In [7]:
from IPython.display import YouTubeVideo
YouTubeVideo('5JnMutdy6Fw')
#https://www.youtube.com/watch?v=5JnMutdy6Fw

In [8]:
# get subcase 1
eig1 = op2.eigenvectors[1]

print(eig1.data_frame)

Mode               1          2          3          4          5          6    \
Freq           8.358      9.507      15.663     20.229     20.306     20.548    
NodeID Item                                                                     
1      t1    5.548e-03  4.671e-06 -1.818e-04 -5.670e-02  1.722e-04 -4.175e-02   
       t2   -2.133e-04  5.699e-03  2.392e-02  5.801e-04 -1.812e-04  1.971e-04   
       t3    8.469e-04  1.512e-03  7.038e-03 -8.160e-03 -1.385e-03 -6.209e-03   
       r1    8.399e-06 -2.241e-04 -1.035e-03 -4.509e-05  6.317e-05 -9.634e-06   
       r2    2.507e-04  1.228e-06 -8.742e-06 -2.571e-03  6.180e-06 -1.767e-03   
       r3    5.261e-05 -1.187e-06 -1.986e-04 -1.310e-04 -2.860e-05 -4.676e-05   
2      t1    5.548e-03  4.671e-06 -1.818e-04 -5.670e-02  1.722e-04 -4.175e-02   
       t2   -1.081e-04  5.696e-03  2.353e-02  3.180e-04 -2.384e-04  1.036e-04   
       t3    3.455e-04  1.510e-03  7.055e-03 -3.018e-03 -1.398e-03 -2.676e-03   
       r1    8.399e-06 -2.24

## Accessing the plate stress/strain
Results are stored on a per element type basis.

The OP2 is the same as an F06, so CQUAD4 elements have centroidal-based results or centroidal-based as well as the results at the 4 corner nodes.

Be careful about what you're accessing.

In [9]:
# element forces/stresses/strains are by element type consistent with the F06, so...
plate_stress = op2.cquad4_stress[1]
print("plate_stress_obj = %s" % type(plate_stress))

# the set of variables in the RealPlateStressArray
print("plate_stress = %s\n" % plate_stress.__dict__.keys())

# list of parameters that define the object (e.g. what is the nonlinear variable name
print("data_code_keys = %s\n" % plate_stress.data_code.keys())

# nonlinear variable name
name = plate_stress.data_code['name']
print("name = %r" % plate_stress.data_code['name'])

print("list-type variables = %s" % plate_stress.data_code['data_names'])

# the special loop parameter
# for modal analysis, it's "modes"
# for transient, it's "times"
# or be lazy and use "_times"
print("modes = %s" % plate_stress.modes) # name + 's'


# extra list-type parameter for modal analysis; see dataNames
#print("mode_cycles =", plate_stress.mode_cycles)

plate_stress_obj = <class 'pyNastran.op2.tables.oes_stressStrain.real.oes_plates.RealPlateStressArray'>
plate_stress = ['_add_new_eid', '_add_new_node', 'subtitle', 'words', 's_code', 'is_built', 'stress_bits', 'load_set', 'itotal', '_add', '_ntotals', 'nnodes', 'isubcase', 'element_name', 'itime', 'nonlinear_factor', 'title', '_times', 'ntotal', 'approach_code', 'sort_bits', 'label', 'element_node', 'is_msc', 'num_wide', 'mode', 'format_code', 'device_code', 'modes', '_times_dtype', 'thermal_bits', 'mode2s', 'data_frame', 'mode2', 'dt', 'data', 'cycle', 'name', 'nelements', 'eigr', 'ielement', 'thermal', 'analysis_code', 'eigrs', 'table_code', 'element_type', 'table_name', 'data_code', 'isTransient', 'sort_code', 'cycles', 'ntimes', 'data_names']

data_code_keys = [u'subtitle', u'stress_bits', u'load_set', u'thermal', u's_code', u'isubcase', u'element_name', u'mode2', u'title', u'approach_code', u'sort_bits', u'label', u'is_msc', u'num_wide', u'format_code', u'device_code', u'_times_d

### Similar to the BDF, we can use object_attributes/methods

In [10]:
#print "attributes =", object_attributes(plate_stress)
print("methods = %s\n" % object_methods(plate_stress))
print('methods2= %s\n' % plate_stress.object_methods())
print("headers = %s\n" % plate_stress.get_headers())

methods = ['apply_data_code', 'approach_code_str', 'build', 'build_dataframe', 'cast_grid_type', 'code_information', 'eid_to_element_node_index', 'get_data_code', 'get_element_index', 'get_element_type', 'get_headers', 'get_nnodes_bilinear', 'get_stats', 'get_unsteady_value', 'is_bilinear', 'is_complex', 'is_curvature', 'is_fiber_distance', 'is_magnitude_phase', 'is_max_shear', 'is_real', 'is_sort1', 'is_sort2', 'is_strain', 'is_stress', 'is_thermal', 'is_von_mises', 'object_attributes', 'object_methods', 'print_data_members', 'print_table_code', 'recast_gridtype_as_string', 'set_table_type', 'update_data_code', 'update_dt', 'write_f06']

methods2= ['apply_data_code', 'approach_code_str', 'build', 'build_dataframe', 'cast_grid_type', 'code_information', 'eid_to_element_node_index', 'get_data_code', 'get_element_index', 'get_element_type', 'get_headers', 'get_nnodes_bilinear', 'get_stats', 'get_unsteady_value', 'is_bilinear', 'is_complex', 'is_curvature', 'is_fiber_distance', 'is_magnit

### Number of Nodes on a CQUAD4
 - For linear CQUAD4s, there is 1 centroidal stress at two locations 
 - For bilinear quads, there are 5 stresses at two locations (4 nodes + centroidal)
 - node_id=0 indicates a centroidal quantity
 - CTRIA3s are always centroidal

#### What sets this?
    STRESS(real, sort1, BILIN) = ALL   # bilinear cquad
    STRESS(real, sort1, CENT) = ALL    # linear quad

    STRAIN(real, sort1, BILIN) = ALL   # bilinear cquad
    STRAIN(real, sort1, CENT) = ALL    # linear quad

#### How do we know if we're bilinear?
```
print("is_bilinear = %s\n" % plate_stress.is_bilinear())
```

#### What locations are chosen?
 That depends on fiber distance/fiber curvature...
 - fiber_curvature - mean stress (oa) & slope (om)
 
    $$ \sigma_{top} = \sigma_{alt} + \frac{t}{2} \sigma_{mean}$$
    
    $$ \sigma_{btm} = \sigma_{alt} + \frac{t}{2} \sigma_{mean}$$
 
 - fiber_distance - upper and lower surface stress (o_top; o_btm)
 - If you have stress, fiber_distance is always returned regardless of your option.

#### What sets this?
    STRAIN(real, sort1, FIBER) = ALL   # fiber distance/default
    STRAIN(real, sort1, STRCUR) = ALL  # strain curvature


#### How do we know if we're using fiber_distance?
```
print("is_fiber_distance = %s" % plate_stress.is_fiber_distance())
```

## Accessing results

In [11]:
# element forces/stresses/strains are by element type consistent 
# with the F06, so...

def abs_max_min(vals):
    absvals = list(abs(vals))
    maxval = max(absvals)
    i = absvals.index(maxval)
    return vals[i]

#-----------------------------
# again, we have linear quads, so two locations per element
print("element_node[:10, :] =\n%s..." % plate_stress.element_node[:10, :])

# lets get the stress for the first 3 CQUAD4 elements
eids = plate_stress.element_node[:, 0]
ueids = np.unique(eids)
print('ueids = %s' % ueids[:3])

# get the first index of the first 5 elements
ieids = np.searchsorted(eids, ueids[:3])
print('ieids = %s' % ieids)

# the easy way to slice data for linear plates
ieids5 = np.vstack([ieids, ieids + 1]).ravel()
ieids5.sort()

print('verify5:\n%s' % ieids5)

#-----------------------------
itime = 0 # static analysis / mode 1
if plate_stress.is_von_mises():  # True
    ovm = plate_stress.data[itime, :, 7]
    print('we have von mises data; ovm=%s\n' % ovm)
else:
    omax_shear = plate_stress.data[itime, :, 7]
    print('we have max shear data; omax_shear=%s\n' % omax_shear)


print("[layer1, layer2, ...] = %s" % ovm[ieids5])

ieid1000 = np.where(eids == 1000)[0]
print('ieid1000 = %s' % ieid1000)
ovm_mode6_eid1000 = ovm[ieid1000]
print("ovm_mode6_eid1000 = %s -> %s" % (ovm_mode6_eid1000, abs_max_min(ovm_mode6_eid1000)))

element_node[:10, :] =
[[1 0]
 [1 0]
 [2 0]
 [2 0]
 [3 0]
 [3 0]
 [4 0]
 [4 0]
 [5 0]
 [5 0]]...
ueids = [1 2 3]
ieids = [0 2 4]
verify5:
[0 1 2 3 4 5]
we have von mises data; ovm=[ 54.222   5.041  13.143 ...,   2.34    6.146   7.368]

[layer1, layer2, ...] = [ 54.222   5.041  13.143  21.222  78.544  17.91 ]
ieid1000 = [1998 1999]
ovm_mode6_eid1000 = [ 90.618  94.09 ] -> 94.0905


In [12]:
# see the difference between "transient"/"modal"/"frequency"-style results
# and "nodal"/"elemental"-style results
# just change imode

imode = 5  # mode 6; could just as easily be dt
iele = 10  # element 10
ilayer = 1

ieid10 = np.where(eids == iele)[0][ilayer]
print('ieid10 = %s' % ieid10)
print(plate_stress.element_node[ieid10, :])


# headers = [u'fiber_distance', u'oxx', u'oyy', u'txy', u'angle', u'omax', u'omin', u'von_mises']
print("ps.modes = %s" % plate_stress.modes[imode])
print("ps.cycles = %s" % plate_stress.cycles[imode])
print("oxx = %s" % plate_stress.data[imode, ieid10, 1])
print("oyy = %s" % plate_stress.data[imode, ieid10, 2])
print("txy = %s" % plate_stress.data[imode, ieid10, 3])
print("omax = %s" % plate_stress.data[imode, ieid10, 5])
print("omin = %s" % plate_stress.data[imode, ieid10, 6])
print("ovm/max_shear = %s" % plate_stress.data[imode, ieid10, 7])

if plate_stress.is_fiber_distance():
    print("fiber_distance = %s" % plate_stress.data[imode, ieid10, 0])
else:
    print("curvature = %s" % plate_stress.data[imode, ieid10, 0])

ieid10 = 19
[10  0]
ps.modes = 6
ps.cycles = 9.80908925027e-45
oxx = -18.8701
oyy = -20.1605
txy = -8.30956
omax = -11.1807
omin = -27.8499
ovm/max_shear = 24.2743
fiber_distance = -0.4


In [13]:
from pyNastran.bdf.bdf import read_bdf
bdf_filename = os.path.abspath(os.path.join(pkg_path, '..', 'models', 'iSat', 'ISat_Launch_Sm_4pt.dat'))
model = read_bdf(bdf_filename)
mass, cg, I = model.mass_properties()

DEBUG:     fname=bdf.pyc                   lineNo=723    ---starting BDF.read_bdf of f:\work\pynastran\pynastran\master3\models\iSat\ISat_Launch_Sm_4pt.dat---
DEBUG:     fname=bdf.pyc                   lineNo=2658   opening 'f:\\work\\pynastran\\pynastran\\master3\\models\\iSat\\ISat_Launch_Sm_4pt.dat'
DEBUG:     fname=bdf.pyc                   lineNo=2658   opening u'f:\\work\\pynastran\\pynastran\\master3\\models\\iSat\\write_modes.v2005'
DEBUG:     fname=cross_reference.pyc       lineNo=527    Cross Referencing...
DEBUG:     fname=bdf.pyc                   lineNo=749    ---finished BDF.read_bdf of f:\work\pynastran\pynastran\master3\models\iSat\ISat_Launch_Sm_4pt.dat---


In [15]:
import getpass
name = getpass.getuser()
os.chdir(os.path.join(r'C:\Users', name, 'Desktop'))

# write the F06 with Real/Imaginary or Magnitude/Phase
# only matters for complex results
#op2.write_f06('isat.f06', is_mag_phase=False)

!head -n 40 isat.f06
gpw = op2.grid_point_weight
print(gpw.object_attributes())

# The mass results are different as pyNastran's mass assumes point masses.
# The larger your model is and the further from the origin, the more accurate
# the result.
# For some applications (e.g. a weight breakdown), this may be fine.
print('cg =\n%s' % gpw.cg)
print('cg = %s' % cg)

                           O U T P U T   F R O M   G R I D   P O I N T   W E I G H T   G E N E R A T O R
0                                                     REFERENCE POINT =        0
                                                                M O
                      *  1.774601E+00  1.402827E-19  2.212874E-19 -1.821217E-17 -3.277270E+01  4.490826E+00 *
                      *  1.402827E-19  1.774601E+00 -4.675622E-19  3.277270E+01 -2.898787E-17 -6.007191E-02 *
                      *  2.212874E-19 -4.675622E-19  1.774601E+00 -4.490826E+00  6.007191E-02 -3.871152E-19 *
                      * -1.821217E-17  3.277270E+01 -4.490826E+00  1.322289E+03  1.414696E+00 -1.250574E+00 *
                      * -3.277270E+01 -2.898787E-17  6.007191E-02  1.414696E+00  1.227074E+03 -2.187713E+02 *
                      *  4.490826E+00 -6.007191E-02 -3.871152E-19 -1.250574E+00 -2.187713E+02  4.272278E+02 *
                                                                 S
                   